In [1]:
# 1. まず*CP.csv からpair、Edisp、 D3のEint、D3なしのEintの組み合わせを得る
import os
import pandas as pd
from IPython.display import display
import glob

# CSVファイルのパス
# CPファイル
csv_file_path = 'ORIENT_RESULT/rac-B2/rac-35Br_93_YI_17mol_3_cis-keto_0@3_cis-keto_dh6_pair_9_CP.csv' 
# dim3ファイル
dim3_file_path = "ORIENT_RESULT/rac-B2/dim3_pair_9.csv" 

data_dir = "/".join(csv_file_path.split("/")[0:-1])
pair_dir = data_dir + "/PAIR"
os.makedirs(pair_dir, exist_ok=True)

# CSVファイルを読み込む
df = pd.read_csv(csv_file_path)
df['Pair'] = df['File Name'].str.extract(r'pair_(\d+)')

# 'Etotal (D3)'を計算するための関数を定義
def calculate_etotal(row, df, col):
    if row['count'] == 1:
        count_2 = df[(df['Pair'] == row['Pair']) & (df['count'] == 2)][col].values[0]
        count_3 = df[(df['Pair'] == row['Pair']) & (df['count'] == 3)][col].values[0]
        return (row[col] - count_2 - count_3)*2625.5
    return None



# 'Etotal (D3)'を計算して新しいカラムに追加
df['Etotal (D3)'] = df.apply(lambda row: calculate_etotal(row, df, 'SCF Done Value (D3-corrected)'), axis=1)
df['Etotal (B3LYP)'] = df.apply(lambda row: calculate_etotal(row, df, 'SCF Done Value (B3LYP)'), axis=1)

# 必要なカラムのみを選択して新しいDataFrameを作成
result_df = df[['Pair', 'Etotal (D3)', 'Etotal (B3LYP)']].dropna()
result_df["Edisp"] = result_df["Etotal (D3)"] - result_df["Etotal (B3LYP)"] 

# 結果を新しいCSVファイルに保存
output_csv_path = data_dir + '/CP_result_processed.csv'  # 出力先のファイルパスを設定してください
result_df.to_csv(output_csv_path, index=False)


In [2]:
# dim3の結果も必要なものを抽出する
df_dim3 = pd.read_csv(dim3_file_path)
df_dim3['Pair'] = df_dim3['File Name'].str.extract(r'pair_(\d+)')
result_df_dim3 = df_dim3[['Pair', 'Electrostatic Energy', 'Induction Energy']]
dim3_output_csv_path = data_dir + '/CP_result_processed_dim3.csv' 
result_df_dim3.to_csv(dim3_output_csv_path, index=False)

In [3]:
# CSVファイルを読み込む
cp_processed_df = pd.read_csv(output_csv_path)
cp_processed_dim3_df = pd.read_csv(dim3_output_csv_path)

# 'Pair'列を基準にして横方向に結合（concatenate）
concat_df = pd.concat([cp_processed_df.set_index('Pair'), cp_processed_dim3_df.set_index('Pair')], axis=1, join='inner').reset_index()
concat_df["Replusion"] = concat_df["Etotal (B3LYP)"] - concat_df["Electrostatic Energy"] - concat_df["Induction Energy"]
concat_df = concat_df[['Pair', 'Electrostatic Energy', 'Induction Energy', 'Replusion', 'Edisp', 'Etotal (D3)', 'Etotal (B3LYP)']]

# 結果を新しいCSVファイルに保存
output_csv_path = data_dir + '/processed_DMP_results.csv'  # 出力先のファイルパスを設定してください
concat_df.to_csv(output_csv_path, index=False)

# ソートしたものを保存
sorted_concat_df = concat_df.sort_values(by='Electrostatic Energy')
sorted_concat_df_path = data_dir + '/processed_DMP_results_sorted.csv'
sorted_concat_df.to_csv(sorted_concat_df_path, index=False)

display(sorted_concat_df)

,Pair,Electrostatic Energy,Induction Energy,Replusion,Edisp,Etotal (D3),Etotal (B3LYP)
12,6,-16.065557,-6.779295,18.586977,-59.120435,-63.378310,-4.257875
11,5,-8.976195,-1.342647,2.903663,-14.120012,-21.535191,-7.415179
10,4,-4.986335,-2.316222,34.269682,-107.530671,-80.563546,26.967125
1,11,-4.573569,-3.594919,11.112647,-29.621786,-26.677627,2.944159
7,1,-3.495479,-0.543469,5.977089,-17.682818,-15.744677,1.938141
5,15,-2.290847,-0.672626,7.011653,-13.534007,-9.485826,4.048180
13,7,-2.013521,-0.273212,4.657687,-9.244881,-6.873927,2.370954
2,12,-2.010818,-0.201693,3.553313,-13.985078,-12.644277,1.340802
8,2,-1.999126,-0.474273,6.234634,-13.553273,-9.792039,3.761235
4,14,-1.821485,-0.424274,2.527209,-9.552799,-9.271349,0.281450


## Groupラベルの追加 (一度目視の確認が必要、上で出力された結果に基づきn_clustersの値を適宜変える)

In [4]:
import pandas as pd
from sklearn.cluster import KMeans

# CSVファイルのパス
csv_file_path = sorted_concat_df_path  

# CSVファイルを読み込む
df = pd.read_csv(csv_file_path)

# KMeansクラスタリングを使用して7つのグループに分ける
kmeans = KMeans(n_clusters=16)
# df['Group'] = kmeans.fit_predict(df[['Electrostatic Energy', 'Induction Energy', 'Replusion', 'Edisp']])
df['Group'] = kmeans.fit_predict(df[['Electrostatic Energy']])

# Groupカラムを'Electrostatic Energy'の大きい順に1から7まで名付ける
group_means = df.groupby('Group')['Electrostatic Energy'].mean().sort_values(ascending=True)
new_group_mapping = {old: new for new, old in enumerate(group_means.index, start=1)}
df['Group'] = df['Group'].map(new_group_mapping)

df = df[['Pair', 'Group', 'Electrostatic Energy', 'Induction Energy', 'Replusion', 'Edisp', 'Etotal (D3)', 'Etotal (B3LYP)']]


# 結果を新しいCSVファイルに保存
grouped_csv_path = data_dir + '/processed_DMP_results_sorted_grouped.csv'  
df.to_csv(grouped_csv_path, index=False)

print("等価であることを確認してください")
display(df)

等価であることを確認してください


,Pair,Group,Electrostatic Energy,Induction Energy,Replusion,Edisp,Etotal (D3),Etotal (B3LYP)
0,6,1,-16.065557,-6.779295,18.586977,-59.120435,-63.378310,-4.257875
1,5,2,-8.976195,-1.342647,2.903663,-14.120012,-21.535191,-7.415179
2,4,3,-4.986335,-2.316222,34.269682,-107.530671,-80.563546,26.967125
3,11,4,-4.573569,-3.594919,11.112647,-29.621786,-26.677627,2.944159
4,1,5,-3.495479,-0.543469,5.977089,-17.682818,-15.744677,1.938141
5,15,6,-2.290847,-0.672626,7.011653,-13.534007,-9.485826,4.048180
6,7,7,-2.013521,-0.273212,4.657687,-9.244881,-6.873927,2.370954
7,12,8,-2.010818,-0.201693,3.553313,-13.985078,-12.644277,1.340802
8,2,9,-1.999126,-0.474273,6.234634,-13.553273,-9.792039,3.761235
9,14,10,-1.821485,-0.424274,2.527209,-9.552799,-9.271349,0.281450


## 中心+group_nのgjfファイル作成 (Groupが等価なことを確認したら)

## PAIRフォルダに"pair_n.gjf"を追加
## PAIRフォルダに"central.gjf"と名付けたファイルを追加、このファイルは中心分子だけの座標を持つファイルでCONSTANT以降の行は消す(改行もすべて)

In [44]:
# 等価なグループのMを持つ行を追加
import shutil
df = pd.read_csv(grouped_csv_path)
groups = df["Group"].unique()
pairs = df["Pair"].unique()
result_path =  pair_dir + "/result"
os.makedirs(result_path, exist_ok=True)
central_file = pair_dir + "/central.gjf"

file_path = pair_dir + "/*.gjf"
paths = [p for p in glob.glob(file_path)]

with open(central_file, 'r') as file:
    lines = file.readlines()

    # 最後の行が空白行でなければ、空白行を追加する
    if lines[-1] != '\n':
        lines.append('\n')

# 変更をファイルに書き戻す
with open(central_file, 'w') as file:
    file.writelines(lines)

for path in paths:
    if 'pair' in path:
        pair_n = int(path.split("pair_")[-1].split(".gjf")[0])
        group_n = df[df["Pair"]==pair_n]["Group"].iloc[-1]
        # コピー先のファイル名を設定
        destination_file = os.path.join(result_path, f'central_+group_{group_n}.gjf')
        # 同じ名前のファイルがすでに存在する場合は処理をスキップ
        if not os.path.exists(destination_file):
            # ファイルをコピー
            shutil.copy(central_file, destination_file)
        else:
            print(f"File '{destination_file}' already exists. Skipping the copy process.")
        
#         pathのfileのM行をdestination_fileの末尾に付け足していく
        with open(path, 'r') as src, open(destination_file, 'a') as dst:
            for line in src:
                if line.endswith(" M\n"):
                    dst.write(line)
